# Como retirar linhas repetidas do header

In [2]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark import SparkContext

In [3]:
spark = SparkSession.builder \
    .master("local") \
    .config("spark.sql.autoBroadcastJoinThreshold", -1) \
    .config("spark.executor.memory", "500mb") \
    .appName("Ex5") \
    .getOrCreate()

22/07/04 16:23:30 WARN Utils: Your hostname, computador resolves to a loopback address: 127.0.1.1; using 10.0.0.135 instead (on interface wlp2s0)
22/07/04 16:23:30 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
22/07/04 16:23:30 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [4]:
df = spark.read.csv('./data/data.csv')

In [5]:
df.show()

+-------+
|    _c0|
+-------+
|Linha 1|
|Linha 2|
|Linha 3|
|     Id|
|      1|
|      2|
|      3|
|      4|
|      5|
+-------+



In [7]:
sc = SparkContext.getOrCreate()

In [8]:
rdd = sc.textFile('./data/data.csv')

## Indexação dos elementos, assim, o primeiro elemento ganha o índicie 0 até o último elemento ganhar índicie 8


In [9]:
rdd.zipWithIndex().collect()

[('Linha 1', 0),
 ('Linha 2', 1),
 ('Linha 3', 2),
 ('Id,Nome,Estado', 3),
 ('1,Pedro,SP', 4),
 ('2,Maria,RJ', 5),
 ('3,Ana,SP', 6),
 ('4,Joao,MG', 7),
 ('5,Lucas,SC', 8)]

## Filtra-se os índicies, coletando apenas os índicies maiores que 2 


In [10]:
rdd.zipWithIndex().filter(lambda x: x[1] > 2).collect()

[('Id,Nome,Estado', 3),
 ('1,Pedro,SP', 4),
 ('2,Maria,RJ', 5),
 ('3,Ana,SP', 6),
 ('4,Joao,MG', 7),
 ('5,Lucas,SC', 8)]

## Uso de map para dividir a tabela de acordo com os índicies da TABELA,
## jogando fora os indicies criados pelo zipwithIndex()


In [11]:
rdd.zipWithIndex().filter(lambda x: x[1] > 2).map(lambda y: y[0]).collect()

['Id,Nome,Estado',
 '1,Pedro,SP',
 '2,Maria,RJ',
 '3,Ana,SP',
 '4,Joao,MG',
 '5,Lucas,SC']

## Separando os elementos no índicie 0 da TABELA do resultado anterior


In [12]:
rdd.zipWithIndex().filter(lambda x: x[1] > 2).map(lambda y: y[0].split(',')).collect()

[['Id', 'Nome', 'Estado'],
 ['1', 'Pedro', 'SP'],
 ['2', 'Maria', 'RJ'],
 ['3', 'Ana', 'SP'],
 ['4', 'Joao', 'MG'],
 ['5', 'Lucas', 'SC']]

In [13]:
rdd_final = rdd.zipWithIndex().filter(lambda x: x[1] > 2).map(lambda y: y[0].split(','))

## Criação do Header


In [14]:
colunas = rdd_final.collect()[0]

In [15]:
colunas

['Id', 'Nome', 'Estado']

## Transformando o rdd em um DataFrame - Problema: há a repetição do header


In [16]:
rdd_final.toDF(colunas).show()

+---+-----+------+
| Id| Nome|Estado|
+---+-----+------+
| Id| Nome|Estado|
|  1|Pedro|    SP|
|  2|Maria|    RJ|
|  3|  Ana|    SP|
|  4| Joao|    MG|
|  5|Lucas|    SC|
+---+-----+------+



In [17]:
linha_pulavel = rdd_final.first()

In [18]:
# Filtração do df, retirando o header repetido

df = rdd_final.filter(lambda x: x != linha_pulavel).toDF(colunas)

In [19]:
df.show()

+---+-----+------+
| Id| Nome|Estado|
+---+-----+------+
|  1|Pedro|    SP|
|  2|Maria|    RJ|
|  3|  Ana|    SP|
|  4| Joao|    MG|
|  5|Lucas|    SC|
+---+-----+------+

